In [ ]:

'''
from lppls import lppls, data_loader
import numpy as np
import pandas as pd
from datetime import datetime as dt
%matplotlib inline

# read example dataset into df 
data = data_loader.nasdaq_dotcom()

# convert time to ordinal
time = [pd.Timestamp.toordinal(dt.strptime(t1, '%Y-%m-%d')) for t1 in data['Date']]

# create list of observation data
price = np.log(data['Adj Close'].values)

# create observations array (expected format for LPPLS observations)
observations = np.array([time, price])

# set the max number for searches to perform before giving-up
# the literature suggests 25
MAX_SEARCHES = 25

# instantiate a new LPPLS model with the Nasdaq Dot-com bubble dataset
lppls_model = lppls.LPPLS(observations=observations)

# fit the model to the data and get back the params
tc, m, w, a, b, c, c1, c2, O, D = lppls_model.fit(MAX_SEARCHES)

# visualize the fit
lppls_model.plot_fit()

# should give a plot like the following...



# compute the confidence indicator
res = lppls_model.mp_compute_nested_fits(
    workers=8,
    window_size=120, 
    smallest_window_size=30, 
    outer_increment=1, 
    inner_increment=5, 
    max_searches=25,
    # filter_conditions_config={} # not implemented in 0.6.x
)

lppls_model.plot_confidence_indicators(res)
# should give a plot like the following...
'''

import numpy as np

import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
from telegram_bot import post_telegram_message,post_telegram_file
from PDFconvert import create_pdf
%matplotlib inline
#import matplotlib.pyplot as plt

import sys
sys.path.append("/home/rizpython236/.virtualenvs/rizenvnew/lib/python3.7/site-packages/")
from lppls import lppls,data_loader  # assuming lppls is installed locally
import yfinance as yf
import time

ticker_symbol = "^GSPC"  # S&P 500
start_date = "1999-07-30"  # start date from the paper
end_date = "2024-04-12"   # end date from the paper

#data = yf.download(ticker_symbol, start=start_date, end=end_date)

def analyze_lppls(data_path="ticker.csv", max_searches=25 , output_path="/home/rizpython236/BT5/screener-outputs/"):
  #def analyze_lppls(data, max_searches=25 , output_path="/home/rizpython236/BT5/screener-outputs/"):
  """
  This function reads NASDAQ data from a CSV file, performs LPPLS analysis, 
  and saves the fit and confidence indicator plots as PNGs.

  Args:
      data_path (str, optional): Path to the CSV file containing NASDAQ data. 
          Defaults to "ticker.csv".
      max_searches (int, optional): Maximum number of searches for the LPPLS model. 
          Defaults to 25.
  """

  # Read the CSV data into a pandas DataFrame
  data = pd.read_csv(data_path)
  #data = data_loader.nasdaq_dotcom()  
  
  # Extract ticker name from filename (assuming filename format like 'abc.csv')
  filename = data_path.split("/")[-1]  # Get the filename from the path
  ticker = filename.split(".")[0]  # Extract the part before the extension (.csv)

  # Assuming the CSV has 'Date' and 'Adj Close' columns
  # Convert 'Date' column to datetime format
  #data['Date'] = pd.to_datetime(data['Date'])
  #time = data['Date'].values 

  # Convert time to ordinal
  time = [pd.Timestamp.toordinal(dt.strptime(t1, '%Y-%m-%d')) for t1 in data['Date']]

  # Create list of observation data (log of Adjusted Close price)
  price = np.log(data['Close'].values)

  # Create observations array
  observations = np.array([time, price])

  # Instantiate a new LPPLS model
  lppls_model = lppls.LPPLS(observations=observations)

  # Fit the model to the data and get back the parameters
  tc, m, w, a, b, c, c1, c2, O, D = lppls_model.fit(max_searches)

  lppls_model.plot_fit()
    
  filename="/home/rizpython236/BT5/screener-outputs/lppls_fit.png"  
  #plt.savefig(filename)
  #time.sleep(1) 
  #post_telegram_file(filename)
  # (Optional) Close the plot window
  #plt.close()    
      

  # Function to save the fit plot as PNG
  def save_fit_plot(model,ticker, output_path):
    plt.figure()
    model.plot_fit()
    plt.savefig(f"{output_path}{ticker}_lppls_fit.png")
    plt.close()

  # Function to save the confidence indicator plot as PNG
  def save_confidence_plot(model, results,ticker, output_path):
    plt.figure()
    model.plot_confidence_indicators(results)
    plt.savefig(f"{output_path}{ticker}_lppls_confidence.png")
    plt.close()

  # Save the fit plot
  #save_fit_plot(lppls_model, ticker, output_path)

  # Compute the confidence indicator
  res = lppls_model.mp_compute_nested_fits(
      workers=8,
      window_size=60, #120, 
      smallest_window_size=12, #30,
      outer_increment=1, 
      inner_increment=5, 
      max_searches=25,
  )
    
  lppls_model.plot_confidence_indicators(res)  

  filename="/home/rizpython236/BT5/screener-outputs/lppls_confidence.png"  
  #plt.savefig(filename)
  #time.sleep(1)
  #post_telegram_file(filename)
  # (Optional) Close the plot window
  #plt.close()    

  # Save the confidence indicator plot
  #save_confidence_plot(lppls_model, res, ticker, output_path)

  print(f"LPPLS model analysis for {ticker} complete. Plots saved to {output_path}.")


# Example usage
#analyze_lppls()  # Use default parameters
# or
output_path="/home/rizpython236/BT5/screener-outputs/"

folder_path='/home/rizpython236/BT5/ticker_15yr/^NSEI.csv'
folder_path1='/home/rizpython236/BT5/ticker_15yr/^CRSLDX.csv'
folder_path2='/home/rizpython236/BT5/ticker_15yr/^NSEMDCP50.csv'
#folder_path='/home/rizpython236/BT5/ticker_daily1yr/^NSEI.csv'

#folder_path='/home/rizpython236/BT5/screener-outputs/'
analyze_lppls(folder_path2, 25, output_path)  # Specify different data path and max_searches

print("completed")



  0%|          | 0/566 [00:00<?, ?it/s]/home/rizpython236/.virtualenvs/rizenvnew/lib/python3.7/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/rizpython236/.virtualenvs/rizenvnew/lib/python3.7/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/rizpython236/.virtualenvs/rizenvnew/lib/python3.7/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/rizpython236/.virtualenvs/rizenvnew/lib/python3.7/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * np.pi)) * np.log((tc - t1) / (tc - t2))
/home/rizpython236/.virtualenvs/rizenvnew/lib/python3.7/site-packages/lppls/lppls.py:617: RuntimeWarning: invalid value encountered in log
  return (w / (2.0 * n

In [ ]:
from lppls import lppls_cmaes,data_loader
data = data_loader.nasdaq_dotcom() 
time = [pd.Timestamp.toordinal(dt.strptime(t1, '%Y-%m-%d')) for t1 in data['Date']]

# Create list of observation data (log of Adjusted Close price)
price = np.log(data['Close'].values)
# Create observations array
observations = np.array([time, price])

# Instantiate a new LPPLS model
#lppls_model = lppls.LPPLS(observations=observations)
    
lppls_model = lppls_cmaes.LPPLSCMAES(observations=observations)
tc, m, w, a, b, c, c1, c2, O, D = lppls_model.fit(max_iteration=2500, pop_size=4)

In [ ]:
1. workers (int, optional):

This parameter specifies the number of worker processes to use for parallel computation. It defaults to 8, but you can adjust it based on your hardware capabilities.
Using multiple workers can significantly speed up the computations, especially for larger datasets. However, keep in mind that increasing the number of workers might also increase memory usage.
2. window_size (int):

This parameter defines the initial window size used for fitting the LPPLS model. It essentially determines the range of data points considered in the first round of fitting.
A larger window size can capture broader trends, while a smaller window size might be better suited for identifying more localized patterns.
3. smallest_window_size (int):

This parameter sets the minimum window size allowed during the nested fitting process. The computation will stop searching for better fits when the window size reaches this value.
This helps prevent excessive computation and potential overfitting, especially when dealing with noisy data.
4. outer_increment (int):

This parameter controls the increment used for increasing the outer window size during the nested fitting process. The outer window size represents the larger timeframe considered for fitting after the initial fit with window_size.
A larger outer_increment will explore a wider range of window sizes more rapidly, while a smaller value will search for better fits more precisely within a narrower range.
5. inner_increment (int):

This parameter defines the increment used for refining the inner window size within each outer window size iteration. The inner window size represents a smaller timeframe nested within the outer window for more localized fitting.
A larger inner_increment will explore different inner window sizes more coarsely within each outer window, while a smaller value will search for optimal inner fits more meticulously.
6. max_searches (int):

This parameter sets the maximum number of searches allowed for finding the best nested fit within each outer window size and inner window size combination.
A higher max_searches value allows for a more exhaustive search for the optimal fit, but it also increases computation time. Conversely, a lower value might miss potentially better fits.
In summary:

These parameters control the trade-off between computational efficiency and finding the best possible nested LPPLS fits for your data. You can experiment with different values to find a suitable configuration based on your dataset size, desired accuracy, and available computational resources.